**<font color = blue size=4>爬取網易雲動態歌詞(更新版)</font>**<br>

In [1]:
import os
import re
import csv
import ast 
import json
import time
import urllib
import random
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent

In [2]:
# 匯入歌手的csv檔案
def import_csv(filepath):
    df_artists = pd.read_csv(filepath, delimiter = ',')
    return df_artists

In [3]:
# 取得歌手對應的歌曲清單，收集歌單內的歌曲ID
# 取得歌曲頁面url，存成list(song_url)
# https://segmentfault.com/a/1190000014948845
def get_songs(artist_id):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    r = requests.get('http://music.163.com/artist?id={}'.format(artist_id),headers=headers)
    song_ids = re.findall(r'song\?id=(\d+?)".+?</a>',r.text) #歌曲id列表
    song_titles = re.findall(r'song\?id=\d+?">(.+?)</a>',r.text)#歌名title列表
    song_url = []
    for i in range(len(song_ids)):
        song_id = song_ids[i]
        url = 'https://music.163.com/#/song?id={}'.format(song_id)
        song_url.append(url)
    
    # 根據特定歌手ID搜尋歌手名稱
    # https://music.163.com/#/artist?id=1045123
    df_artists_filter = df_artists.loc[df_artists["artist_id"] == artist_id]
    artist_name = df_artists_filter['artist_name'].values[0]

    artist_url = 'http://music.163.com/artist?id=' + str(artist_id) #歌手頁面連結
    
    return [song_ids, song_titles, song_url, artist_name, artist_url]

In [4]:
# 歌單的歌轉成dataframe並存成csv
# 亂碼使用encoding='utf_8_sig'解決
def get_songinfo(song_ids, song_titles, song_url, artist_name):
    df_songs = pd.DataFrame()
    artist_name_list = []
    for i in range(len(song_ids)):
        artist_name_list.append(artist_name)
    df_songs['Artist'] = pd.Series(artist_name_list)
    df_songs['SongID'] = pd.Series(song_ids)
    df_songs['SongTitle'] = pd.Series(song_titles)
    df_songs['SongURL'] = pd.Series(song_url)
    artist_name_new = artist_name.replace(' / ',' ')
    artist_name_new = validateTitle(artist_name_new)
    filepath = 'CloudMusic_csv/artists/df_songs_' + artist_name_new + '.csv'
    df_songs.to_csv(filepath, encoding='utf_8_sig')
    return df_songs

In [5]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, "", title)  # 替換為空白
    return new_title

In [6]:
# 取得歌名和歌手
def get_name_artist(df_songs, count):
    artistname = df_songs['Artist'][count]
    artistname = validateTitle(artistname)
    songname = df_songs['SongTitle'][count]
    songname = validateTitle(songname)
    songid = df_songs['SongID'][count]
    songid = validateTitle(songid)
    #print(artistname + '_' + songname + '\n' + songid)
    return artistname, songname, songid

In [7]:
# 取得動態歌詞
def get_lyrics(songid):
    headers={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
             'Accept-Encoding': 'gzip, deflate',
             'Accept-Language': 'zh-CN,zh;q=0.9',
             'Connection': 'keep-alive',
             'Cookie': '_iuqxldmzr_=32; _ntes_nnid=0e6e1606eb78758c48c3fc823c6c57dd,1527314455632; '
                       '_ntes_nuid=0e6e1606eb78758c48c3fc823c6c57dd; __utmc=94650624; __utmz=94650624.1527314456.1.1.'
                       'utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); WM_TID=blBrSVohtue8%2B6VgDkxOkJ2G0VyAgyOY;'
                       ' JSESSIONID-WYYY=Du06y%5Csx0ddxxx8n6G6Dwk97Dhy2vuMzYDhQY8D%2BmW3vlbshKsMRxS%2BJYEnvCCh%5CKY'
                       'x2hJ5xhmAy8W%5CT%2BKqwjWnTDaOzhlQj19AuJwMttOIh5T%5C05uByqO%2FWM%2F1ZS9sqjslE2AC8YD7h7Tt0Shufi'
                       '2d077U9tlBepCx048eEImRkXDkr%3A1527321477141; __utma=94650624.1687343966.1527314456.1527314456'
                       '.1527319890.2; __utmb=94650624.3.10.1527319890',
             'Host': 'music.163.com',
             'Referer': 'http://music.163.com/',
             'Upgrade-Insecure-Requests': '1',
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/66.0.3359.181 Safari/537.36'}
    url = 'http://music.163.com/api/song/lyric?id={}&lv=-1&kv=-1&tv=-1'.format(songid)
    user_agent = generate_user_agent()
    response = requests.get(url, headers={ 'user-agent': user_agent })
    # 隨機暫停 1~5 秒
    time.sleep(random.uniform(1,5))
    data = response.json()
    return data

In [8]:
# 下載動態歌詞，取代動態歌詞中的「’」符號，判斷回傳的lrc是否為空
# 解決UnicodeDecodeError: 'cp950'問題，加入encoding="utf-8"
def save_lrc_file(data, artistname, songname, count):
    content = data['lrc']['lyric']
    # lrc不為空
    if content != "":
        content = data['lrc']['lyric'].replace("’", "'")
        songname_new = songname.replace(' / ',' ')
        filepath = 'CloudMusic_lrc/artists/'+ artist_name + '_' + songname_new +'.lrc'
        #檔名重複篩選
        if os.path.isfile(filepath):
            #檔案存在
            pass
        else:
            #檔案不存在
            with open(filepath, 'w', encoding="utf-8") as file:
                file.write(content)
            print(filepath)
            count += 1
    # lrc為空
    else:
        pass
    return count

In [11]:
# 主程式抓歌曲
count_list = []
base = 2000 #base,base+100
for w in range(base,base+100):
    print('Round:' + str(w))
    #artist_id = '45236'
    df_artists = import_csv('CloudMusic_csv/df_artists.csv')
    artist_id = df_artists['artist_id'][w]
    [song_ids, song_titles, song_url, artist_name, artist_url] = get_songs(artist_id)
    df_songs = get_songinfo(song_ids, song_titles, song_url, artist_name)
    count = 0
    for i in range(len(df_songs)):
        artistname, songname, songid = get_name_artist(df_songs, i)
        try:
            data = get_lyrics(songid)
            count = save_lrc_file(data, artistname, songname, count)    
        except:
            pass
    
    count_list.append(count)
    print('Artist: ' + artistname + ' / Count: ' + str(count))
    print('-'*80)

Round:2000
CloudMusic_lrc/artists/Unlike Pluto_Wannabe.lrc
CloudMusic_lrc/artists/Unlike Pluto_Everything Black (feat. Mike Taylor).lrc
CloudMusic_lrc/artists/Unlike Pluto_We Will Rock You.lrc
CloudMusic_lrc/artists/Unlike Pluto_No Scrubs.lrc
CloudMusic_lrc/artists/Unlike Pluto_Waiting For You (feat. Joanna Jones).lrc
CloudMusic_lrc/artists/Unlike Pluto_I Need A Win.lrc
CloudMusic_lrc/artists/Unlike Pluto_Worst In Me.lrc
CloudMusic_lrc/artists/Unlike Pluto_Closure (Year 09).lrc
CloudMusic_lrc/artists/Unlike Pluto_Happy Together.lrc
CloudMusic_lrc/artists/Unlike Pluto_Everything Black .lrc
CloudMusic_lrc/artists/Unlike Pluto_Western Love (Pluto Tapes).lrc
CloudMusic_lrc/artists/Unlike Pluto_Now I Don't Care.lrc
CloudMusic_lrc/artists/Unlike Pluto_Can't hold us (Unlike Pluto remix).lrc
CloudMusic_lrc/artists/Unlike Pluto_Fade All My Life.lrc
CloudMusic_lrc/artists/Unlike Pluto_Look At Me.lrc
CloudMusic_lrc/artists/Unlike Pluto_Searching For You (Original Mix).lrc
CloudMusic_lrc/artists/U

CloudMusic_lrc/artists/Uzeyir Mehdizade_Sene Hele Buda Azdi.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Mensiz Xosbextsen.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Uzaq Dur (2017 Albom).lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Dengi.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Yarimnan Ayrildim.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Sehere Qeder.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Nostalji Mix.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Getirmeyin Onu Yanima.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Teki Sen Xosbext Olasan Yarim.lrc
CloudMusic_lrc/artists/Uzeyir Mehdizade_Cavab Ver.lrc
Artist: Uzeyir Mehdizade / Count: 30
--------------------------------------------------------------------------------
Round:2004
CloudMusic_lrc/artists/Udit Narayan_Aankhen Khuli.lrc
CloudMusic_lrc/artists/Udit Narayan_Humko Humise Chura Lo.lrc
CloudMusic_lrc/artists/Udit Narayan_Bole Chudiyan.lrc
CloudMusic_lrc/artists/Udit Narayan_Jaadu Teri Nazar.lrc
CloudMusic_lrc/artists/Udit Nara

CloudMusic_lrc/artists/Uwe Kröger_Liebe endet nie (aus Der Besuch der alten Dame).lrc
CloudMusic_lrc/artists/Uwe Kröger_Wenn Ich Dein Spiegel Wär.lrc
CloudMusic_lrc/artists/Uwe Kröger_Prolog Tod.lrc
CloudMusic_lrc/artists/Uwe Kröger_Opening.lrc
CloudMusic_lrc/artists/Uwe Kröger_Schwarzer Engel.lrc
CloudMusic_lrc/artists/Uwe Kröger_Sie Passt Nicht.lrc
CloudMusic_lrc/artists/Uwe Kröger_Szenen Einer Ehe.lrc
CloudMusic_lrc/artists/Uwe Kröger_Die Fäden in der Hand.lrc
CloudMusic_lrc/artists/Uwe Kröger_Zwischen Traum und Wirklichkeit.lrc
CloudMusic_lrc/artists/Uwe Kröger_Edelweiss.lrc
CloudMusic_lrc/artists/Uwe Kröger_Boulevard der Sehnsucht.lrc
CloudMusic_lrc/artists/Uwe Kröger_Ich denk an Dich.lrc
CloudMusic_lrc/artists/Uwe Kröger_Siebzehn Stufen.lrc
CloudMusic_lrc/artists/Uwe Kröger_Bse Jungs.lrc
CloudMusic_lrc/artists/Uwe Kröger_Flieh mit mir.lrc
CloudMusic_lrc/artists/Uwe Kröger_Atlantics.lrc
CloudMusic_lrc/artists/Uwe Kröger_Requiem.lrc
CloudMusic_lrc/artists/Uwe Kröger_Willkommen im N

CloudMusic_lrc/artists/Unique_Goddess.lrc
CloudMusic_lrc/artists/Unique_Brooklyn to You.lrc
CloudMusic_lrc/artists/Unique_Beat of My Drum.lrc
CloudMusic_lrc/artists/Unique_Wherever You Are (feat. Charice).lrc
CloudMusic_lrc/artists/Unique_In Your Eyes.lrc
CloudMusic_lrc/artists/Unique_Only Forever.lrc
CloudMusic_lrc/artists/Unique_Cliche.lrc
CloudMusic_lrc/artists/Unique_Tonight.lrc
CloudMusic_lrc/artists/Unique_Foundation.lrc
CloudMusic_lrc/artists/Unique_Friend.lrc
Artist: Unique / Count: 11
--------------------------------------------------------------------------------
Round:2030
CloudMusic_lrc/artists/uǝǝʇʇɐǝqpɐǝp_summer rain.lrc
CloudMusic_lrc/artists/uǝǝʇʇɐǝqpɐǝp_The break up.lrc
Artist: uǝǝʇʇɐǝqpɐǝp / Count: 2
--------------------------------------------------------------------------------
Round:2031
CloudMusic_lrc/artists/Uncle Kracker_My Hometown.lrc
CloudMusic_lrc/artists/Uncle Kracker_Drift Away.lrc
CloudMusic_lrc/artists/Uncle Kracker_Smile.lrc
CloudMusic_lrc/artists/Uncle

CloudMusic_lrc/artists/Ulrik Munther_Alburn Road.lrc
CloudMusic_lrc/artists/Ulrik Munther_The Box.lrc
CloudMusic_lrc/artists/Ulrik Munther_Heroes In Defeat (Change Your Mind).lrc
CloudMusic_lrc/artists/Ulrik Munther_Life.lrc
CloudMusic_lrc/artists/Ulrik Munther_Soldiers.lrc
CloudMusic_lrc/artists/Ulrik Munther_Förråda dig.lrc
CloudMusic_lrc/artists/Ulrik Munther_Tell the World I'm Here.lrc
CloudMusic_lrc/artists/Ulrik Munther_Glad I Found You.lrc
CloudMusic_lrc/artists/Ulrik Munther_Requiem.lrc
Artist: Ulrik Munther / Count: 17
--------------------------------------------------------------------------------
Round:2051
CloudMusic_lrc/artists/Underscores _Taiyaki.lrc
Artist: Underscores  / Count: 1
--------------------------------------------------------------------------------
Round:2052
Artist: Uverlaw / Count: 0
--------------------------------------------------------------------------------
Round:2053
CloudMusic_lrc/artists/Ugly Duckling_Left Behind.lrc
CloudMusic_lrc/artists/Ugly Du

Artist: Ultimate Dance Hits / Count: 0
--------------------------------------------------------------------------------
Round:2070
CloudMusic_lrc/artists/UndreamedPanic_New King.lrc
CloudMusic_lrc/artists/UndreamedPanic_Close Your Eyes.lrc
CloudMusic_lrc/artists/UndreamedPanic_The Storm.lrc
CloudMusic_lrc/artists/UndreamedPanic_The Quest For Kindness.lrc
CloudMusic_lrc/artists/UndreamedPanic_Flying Higher.lrc
CloudMusic_lrc/artists/UndreamedPanic_Style.lrc
CloudMusic_lrc/artists/UndreamedPanic_The One You Should Know.lrc
Artist: UndreamedPanic / Count: 7
--------------------------------------------------------------------------------
Round:2071
CloudMusic_lrc/artists/Ufo361_Wir sind Kral.lrc
CloudMusic_lrc/artists/Ufo361_YKKE.lrc
CloudMusic_lrc/artists/Ufo361_Pass auf wen du liebst.lrc
CloudMusic_lrc/artists/Ufo361_Shot.lrc
CloudMusic_lrc/artists/Ufo361_Richard Millie.lrc
CloudMusic_lrc/artists/Ufo361_On Time.lrc
CloudMusic_lrc/artists/Ufo361_0430.lrc
CloudMusic_lrc/artists/Ufo361_Nike

Artist: UNOMAS / Count: 2
--------------------------------------------------------------------------------
Round:2093
CloudMusic_lrc/artists/Uner_Positive.lrc
Artist: Uner / Count: 1
--------------------------------------------------------------------------------
Round:2094
CloudMusic_lrc/artists/Umit Sayin_Hep Senin Yüzünden.lrc
CloudMusic_lrc/artists/Umit Sayin_Biz Kopamayız.lrc
Artist: Umit Sayin / Count: 2
--------------------------------------------------------------------------------
Round:2095
CloudMusic_lrc/artists/Udex_Left Behind (Original Mix).lrc
Artist: Udex / Count: 1
--------------------------------------------------------------------------------
Round:2096
CloudMusic_lrc/artists/USAI_Hypnotize (Acapella) .lrc
CloudMusic_lrc/artists/USAI_Sad Story (Out Of Luck) (USAI & Crusy Remix).lrc
CloudMusic_lrc/artists/USAI_405 (USAI Remix).lrc
Artist: USAI / Count: 3
--------------------------------------------------------------------------------
Round:2097
Artist: Uli Jon Roth / 

In [12]:
# 有下載到的lrc數量
print(count_list)
print('LRC count:',len(count_list))

# 把每一位歌手抓到的lrc儲存起來並寫入csv
with open("CloudMusic_csv/df_songcount.csv", "a", encoding="utf-8", newline='') as csvfile:
    writer = csv.writer(csvfile)
    Round = str(base) + '-' + str(base+100)
    Count_list = count_list
    #第一次寫入
    #writer.writerow(['Round', 'Count_list'])
    #之後寫入
    writer.writerow([Round, Count_list])

df_songcount = pd.read_csv('CloudMusic_csv/df_songcount.csv', delimiter = ',')
df_songcount

[48, 23, 48, 30, 38, 15, 4, 3, 29, 0, 5, 2, 33, 0, 31, 0, 1, 4, 7, 5, 1, 35, 0, 0, 3, 1, 0, 6, 3, 11, 2, 20, 4, 0, 1, 0, 1, 19, 0, 0, 3, 1, 2, 8, 10, 5, 2, 0, 0, 4, 17, 1, 0, 27, 0, 37, 9, 0, 2, 1, 0, 0, 1, 4, 11, 1, 0, 0, 6, 0, 7, 20, 0, 1, 0, 15, 3, 0, 1, 29, 1, 0, 2, 2, 0, 1, 0, 0, 1, 1, 2, 6, 2, 1, 2, 1, 3, 0, 1, 2]
LRC count: 100


,Round,Count_list
0,0-100,"[36, 44, 24, 5, 50, 38, 6, 43, 32, 40, 50, 26,..."
1,100-200,"[46, 45, 31, 12, 38, 39, 21, 46, 5, 46, 30, 18..."
2,200-300,"[49, 46, 49, 49, 28, 34, 5, 49, 9, 24, 49, 16,..."
3,300-400,"[25, 47, 48, 37, 47, 30, 38, 48, 50, 29, 1, 32..."
4,400-500,"[50, 50, 20, 6, 5, 6, 27, 1, 49, 3, 48, 34, 17..."
5,500-600,"[46, 27, 14, 46, 24, 10, 25, 20, 1, 6, 29, 13,..."
6,600-700,"[50, 48, 45, 43, 49, 3, 43, 49, 5, 22, 1, 30, ..."
7,700-800,"[37, 6, 24, 19, 21, 7, 41, 2, 1, 44, 9, 3, 8, ..."
8,800-900,"[49, 9, 2, 5, 6, 6, 27, 36, 19, 50, 1, 20, 35,..."
9,900-1000,"[49, 46, 47, 49, 39, 12, 48, 48, 14, 48, 22, 5..."


In [62]:
url = 'https://m801.music.126.net/20200113025447/8cd60d138777a13dd24f33d64be06b59/jdyyaac/520f/0158/0308/eefd2984f862ac2408b94bb0011ebcfa.m4a'
response = requests.get(url)
content = response.content
save_mp3_file('Alan Walker_Faded.mp3', content)

CloudMusic_mp3/Alan Walker_Faded.mp3


In [60]:
# 儲存音樂
def save_mp3_file(filename, content):
    filepath = 'CloudMusic_mp3/'+ filename
    with open(filepath, mode = "wb") as f:
        f.write(content)
    print(filepath)

In [ ]:
# 下載音樂.mp3
def download_music(songname, songid):
    # 取得Request URL
    url = 'http://music.163.com/song/media/outer/url?id=' + str(songid)+'.mp3'
    response = requests.get(url)
    content = response.content
    # mp3檔案命名規則
    save_mp3_file(songname + '_' + songid + '.mp3', content)